## Temporary ICD Update Notebook
This is to be used to update an ICD telecommand dictionary csv sheet from a previous ICD iteration (ex. from a previous PIMS test) to include any newly added commands while still preserving any definitions present in the last dictionary. This is not to be used for generating any official ICDs to be used during mission since telecommand severities and definitions may not be up-to-date (or even correct) since that's not completely necessary where these interim ICDs are needed while FSW is still flushing out its command set.

Note: this only adds new command names and doesn't remove anything old since that's not necessary (an possibly undesirable) for this application.

Note also: this script is really dumb and doesn't recognize when commands have simply had their names changed but are otherwise the same old commands.

In [9]:
# SETTINGS:
i: int = 4
last_icd_path = f'./test-data/ICD_telecommands_{i:d}.csv'
new_icd_path=f'./test-data/ICD_telecommands_{i+1:d}.csv'

# Map from column names to IDs in ICD:
start_row = 2 # index of the first row with actual data (after any sheet and column headers)
cols = {
    'opcode': 0,
    'hazard': 1,
    'yamcs_name': 2,
    'description': 3,
    'usage': 4
}

In [10]:
import csv

from IrisBackendv3.data_standards import DataStandards
from IrisBackendv3.data_standards.prebuilt import add_to_standards, watchdog_heartbeat_tvac
from IrisBackendv3.data_standards.logging import logger as DsLogger

In [3]:
DsLogger.setLevel('CRITICAL')
standards = DataStandards.build_standards()
add_to_standards(standards, watchdog_heartbeat_tvac)
standards.print_overview()

Data Standards Overview: [

	Module[256]::BlockDriver

		Commands:

		Telemetry:

			0.	Channel[0]::BdCycles: uint32

		Events:

	Module[512]::RateGroupDriver

		Commands:

		Telemetry:

		Events:

	Module[768]::ActiveRateGroup-RateGroupLowFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1024]::ActiveRateGroup-RateGroupMedFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1280]::ActiveRateGroup-RateGroupHiFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1536]::CubeRoverTime

		Commands:

		Teleme

In [11]:
# Load last ICD:
icd = []
with open(last_icd_path, mode='r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        icd.append(row)

In [12]:
# Grab all commands from ICD:
old_cmds = [(entry[cols['opcode']].lower(), entry[cols['yamcs_name']]) for entry in icd[start_row:] if entry[cols['yamcs_name']] != '']

# Grab all commands in the latest data standards:
current_cmds = [(f'0x{(mid | c.ID):04x}', n) for n, (mid, c) in standards.commands_chainmap_name.items()]
# Double up all commands (like in icd):
current_cmds = [x for op,n in current_cmds for x in ((op, n+'_wired'), (op, n+'_wifi'))]


# Grab list of all new command names (in current standards but not the old list):
new_cmds = [(op,n) for (op,n) in current_cmds if n not in [n for _,n in old_cmds]]


# Also check if there are any old commands with a mismatched opcodes:
bad_ops = [(op,n) for (op,n) in current_cmds if (op,n) not in new_cmds and (op,n) not in old_cmds]
bad_ops = [([old_op for (old_op,no) in old_cmds if no == n][0], new_op, n) for (new_op,n) in bad_ops]
if len(bad_ops) > 1:
    print("NOTE: The following opcodes have changed (old_op, current_op, n). The current (latest) opcodes will be used.")
    print(bad_ops)

In [13]:
# Update old command opcodes in ICD:
for (_, new_op, n) in bad_ops:
    i, row = [(i, row) for (i, row) in enumerate(icd) if row[cols['yamcs_name']] == n][0]
    row[cols['opcode']] = new_op
    icd[i] = row

# Append all newly added commands to icd:
for opcode, name in new_cmds:
    # Create an empty set of columns:
    entry = [''] * len(cols)
    # Populate relevant columns:
    entry[cols['opcode']] = opcode
    entry[cols['yamcs_name']] = name
    icd.append(entry)

In [7]:
# Write out the new ICD:
with open(new_icd_path, mode='w') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"')
    for row in icd:
        writer.writerow(row)